## Initialize the libraries and origin and destination workspaces

Run the following commands on the shell
```
sudo ln -s /anaconda/envs/py35/bin/pip /usr/bin/pip3
pip3 install azure-cognitiveservices-vision-customvision
```

In [1]:
from azure.cognitiveservices.vision.customvision.training import training_api
from azure.cognitiveservices.vision.customvision.training.models import ImageUrlCreateEntry, Region

# DESTINATION Keys
dest_training_key = "970bf90c200d4f429a233dee4239e8f1"
dest_prediction_key = "23287e52d75349138403089743a504ab"
dest_project_new_name = "PowerItems_Detection_Copy"

# ORIGIN Keys
training_key = "970bf90c200d4f429a233dee4239e8f1"
prediction_key = "23287e52d75349138403089743a504ab"

trainer = training_api.TrainingApi(training_key)
dest_trainer = training_api.TrainingApi(dest_training_key)

# Find the object detection domain
obj_detection_domain = next(domain for domain in trainer.get_domains() if domain.type == "ObjectDetection")
dest_obj_detection_domain = next(domain for domain in dest_trainer.get_domains() if domain.type == "ObjectDetection")

## Get the origin project ID reference

In [2]:
myProjects = trainer.get_projects()

In [3]:
for project in myProjects:
    print(project.name)
    print(project.id)
    print(project.description)

PowerItems_Detection
70e7718a-e1f1-45d5-bba1-60b224f55365
Model to detect Power line objects


In [4]:
Project = trainer.get_project(project_id="70e7718a-e1f1-45d5-bba1-60b224f55365")

## Create the destination Project 

In [5]:
dest_Project = dest_trainer.create_project(dest_project_new_name, domain_id=dest_obj_detection_domain.id)

In [6]:
for project in dest_trainer.get_projects():
    print(project.name)
    print(project.id)
    print(project.description)

PowerItems_Detection_Copy
3b7d7caa-932f-4b1c-ab0d-8529a9ed3652

PowerItems_Detection
70e7718a-e1f1-45d5-bba1-60b224f55365
Model to detect Power line objects


## Get the tags on origin project and create same tags on destination project

In [7]:
dest_tags = []
for tag in trainer.get_tags(Project.id):
    dest_tags.append(dest_trainer.create_tag(dest_Project.id, tag.name))
    print(tag.name)

crossarm
pole
insulator
switch
transformer


In [8]:
dest_tags_dict = {}
dest_tag_ids = []
for tag in dest_tags:
    dest_tags_dict[tag.name] = tag.id
    dest_tag_ids.append(tag.id)

print(dest_tags_dict)
print(dest_tag_ids)

{'crossarm': '3cc6b690-834b-45eb-8584-592ebe6632cc', 'insulator': '02c861dd-af67-49e3-8720-38d11ed8d450', 'switch': 'd07557e2-b358-478f-a37e-cf1ab2477c8c', 'pole': '2d33198d-5317-4e05-91fc-73eac201d036', 'transformer': '153853d6-941c-4adb-8008-c4618bbe1aa6'}
['3cc6b690-834b-45eb-8584-592ebe6632cc', '2d33198d-5317-4e05-91fc-73eac201d036', '02c861dd-af67-49e3-8720-38d11ed8d450', 'd07557e2-b358-478f-a37e-cf1ab2477c8c', '153853d6-941c-4adb-8008-c4618bbe1aa6']


## Get the images with regions on origin project

In [9]:
tagged_images = trainer.get_tagged_images(Project.id, take=trainer.get_tagged_image_count(Project.id))

In [10]:
print ("Adding images...")
tagged_images_with_regions = []
for image in tagged_images:
    dest_regions = []
    
    for region in image.regions:
        dest_regions.append(Region(tag_id=dest_tags_dict[region.tag_name], left=region.left, top=region.top, width=region.width, height=region.height))

    tagged_images_with_regions.append(ImageUrlCreateEntry(url=image.image_uri, tag_ids=dest_tag_ids, regions=dest_regions))
print("Done")

Adding images...
Done


## Create the images with regions on destination project

In [11]:
limit = 64 # this is a limit imposed on the API, so we need to batch the creation process
count_of_images = len(tagged_images_with_regions)

for i in range(0,count_of_images,limit):
    begin=i
    end=limit+i
    if(end > count_of_images ): end = count_of_images
    dest_trainer.create_images_from_urls(dest_Project.id, images=tagged_images_with_regions[begin:end])

In [12]:
print("Count of Tagged images on destination project: " + str(trainer.get_tagged_image_count(Project.id)))
print("Count of Tagged images on destination project: " + str(dest_trainer.get_tagged_image_count(dest_Project.id)))

Count of Tagged images on destination project: 220
Count of Tagged images on destination project: 220


## Train the model on destination Project

In [13]:
import time

print ("Training... (showing status each 10 seconds)")
iteration = dest_trainer.train_project(dest_Project.id)
while (iteration.status != "Completed"):
    iteration = dest_trainer.get_iteration(dest_Project.id, iteration.id)
    print ("Training status: " + iteration.status)
    time.sleep(10)

# The iteration is now trained. Make it the default project endpoint
dest_trainer.update_iteration(dest_Project.id, iteration.id, is_default=True)
print ("Done!")

Training... (showing status each 10 seconds)
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Training
Training status: Completed
Done!


In [22]:
performance = dest_trainer.get_iteration_performance(dest_Project.id, iteration.id, threshold=0.15, overlap_threshold=0.30)
print("Iteration: " + iteration.name)
print("Precision: " + str(performance.precision * 100)+ "%")
print("Recall: " + str(performance.recall * 100) + "%")

Iteration: Iteration 1
Precision: 69.78417%
Recall: 44.2922384%


In [23]:
per_tag_performance = performance.per_tag_performance

In [30]:
for tag in per_tag_performance:
    print ("Tag: " + tag.name + "\t\t" + "Precision: " + str(tag.precision) + "\t" + "Recall: " + str(tag.recall))

Tag: switch		Precision: 0.5882353	Recall: 0.4
Tag: insulator		Precision: 0.61403507	Recall: 0.333333343
Tag: pole		Precision: 0.8181818	Recall: 0.5625
Tag: crossarm		Precision: 0.6666667	Recall: 0.4117647
Tag: transformer		Precision: 0.909090936	Recall: 0.8695652


## Get and use the default prediction endpoint

In [14]:
#You can make a web search online for "powerline pole" and get an URL:
url = "https://qtxasset.com/styles/breakpoint_xl_880px_w/s3fs/FierceTelecom-1512405505/power-line.jpg/power-line.jpg"

<div style="width:100%; margin-left:auto; margin-right:auto; margin-bottom:5px; margin-top:17px;">
<img src="https://qtxasset.com/styles/breakpoint_xl_880px_w/s3fs/FierceTelecom-1512405505/power-line.jpg/power-line.jpg" alt="IMAGE" /><br>
</div>

In [15]:
from azure.cognitiveservices.vision.customvision.prediction import prediction_endpoint
from azure.cognitiveservices.vision.customvision.prediction.prediction_endpoint import models

# Now there is a trained endpoint that can be used to make a prediction
dest_predictor = prediction_endpoint.PredictionEndpoint(dest_prediction_key)

# Open the sample image and get back the prediction results.
results = dest_predictor.predict_image_url(dest_Project.id, iteration_id=iteration.id, url=url)

In [16]:
# Display the results.
for prediction in results.predictions:
    if(prediction.probability > 0.2): # only show the prediction with more than 30% threshold probability
        print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100), prediction.bounding_box.left, prediction.bounding_box.top, prediction.bounding_box.width, prediction.bounding_box.height)

	crossarm: 24.25% 0.08148 0.12148001 0.5689244 0.207973123
	insulator: 28.85% 0.5895231 0.131308734 0.0326781869 0.09359963
	insulator: 31.83% 0.193739489 0.484787375 0.054199174 0.107655793
	switch: 33.51% 0.08172339 0.110237442 0.06170848 0.09260795
	switch: 35.86% 0.213801533 0.527242661 0.0508965254 0.139448941
